In [ ]:
import matplotlib.pyplot as plt

def plot_coefficients(title, coefficients):
    ll, (lh, hl, hh) = coefficients
    
    rows_r = 2
    columns_c = 2
    
    fig = plt.figure(figsize=(7, 7))
    plt.title(title)
    
    ax = fig.add_subplot(rows_r, columns_c, 1)
    ax.set_title('LL')
    plt.imshow(ll)
    
    ax = fig.add_subplot(rows_r, columns_c, 2)
    ax.set_title('LH')
    plt.imshow(lh)
    
    ax = fig.add_subplot(rows_r, columns_c, 3)
    ax.set_title('HL')
    plt.imshow(hl)
    
    ax = fig.add_subplot(rows_r, columns_c, 4)
    ax.set_title('HH')
    plt.imshow(hh)

In [ ]:
import pywt
import numpy as np

image_file = 'avu.png'
file_path = f'data/images/{image_file}'
rows = 8
cols = 8  
chunk_size = rows * cols  
print(pywt.wavelist())
wavelet = 'bior2.2'
count = 0
for wavelet in pywt.wavelist():
    try:
        with open(file_path, 'rb') as file:
            while True:
                chunk = file.read(chunk_size)
                if not chunk:
                    break  # end of file
        
                remaining_bytes = chunk_size - (len(chunk) % chunk_size)
                if remaining_bytes < chunk_size:
                    padding = bytearray([0] * remaining_bytes)
                    chunk += padding
                
                chunk_uint8 = np.frombuffer(chunk, dtype=np.uint8)
                chunk_uint8_2d = chunk_uint8.reshape((-1, cols))
                # print(chunk_uint8_2d)
                
                
                coeffs = pywt.wavedec2(chunk_uint8_2d, wavelet=wavelet, level=2)
        
                LL2, (LH2, HL2, HH2), (LH1, HL1, HH1) = coeffs
                
                
                rec_chunk = np.uint8(pywt.waverec2(coeffs=coeffs, wavelet=wavelet))
                print(wavelet)
                print(LH1.size,HL1.size, HH1.size, LL2.size)
                print()
                # print(np.uint8(rec_chunk))   
                count += 1
                # print(count)
                break
    except:
        print(f'{wavelet} failed to load')


In [64]:
from math import floor, ceil
import numpy as np
import random

def iwt53(c):    
    s = c[0::2]
    d = c[1::2]
    l = len(s)

    a = d[0:l-1] - np.floor(0.5*(s[0:l-1]+s[1:l])) 
    b = d[l-1] - s[l-1] 
    d = np.concatenate((a, b), axis=None)   

    a = s[0] + np.floor(0.5*d[0] + 0.5)
    b = s[1:l] + np.floor(0.25*(d[1:l] + d[0:l-1]) + 0.5)
    s = np.concatenate((a, b), axis=None)
    
    return s, d


def iiwt53(s,d):
    l = len(s)

    a = s[0] - np.floor(0.5*d[0] + 0.5)
    b = s[1:l] - np.floor(0.25*(d[1:l] + d[0:l-1]) + 0.5)
    s = np.concatenate((a, b), axis=None)

    a = d[0:l-1] + np.floor(0.5*(s[0:l-1]+s[1:l])) 
    b = d[l-1] + s[l-1] 
    d = np.concatenate((a, b), axis=None)   

    c2 = np.column_stack((s,d)).ravel()

    return c2


def main():    
    c = np.array([random.randint(0, 100) for x in range(4096)])    
    c = np.arange(64)
    s,d = iwt53(c)
    print(f's: {s}\nd: {d}')
    c2 = iiwt53(s,d)

    if np.array_equal(c, c2):
        print("perfect reconstruction...")

if __name__ == "__main__":
    main()

s: [ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34.
 36. 38. 40. 42. 44. 46. 48. 50. 52. 54. 56. 58. 60. 62.]
d: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1.]
perfect reconstruction...


In [62]:
# Lifting scheme version of the discrete wavelet transform in JPEG 2000

# The approximation and detail coefficients are interleaved in the output of dwt (even 
# indices for approx, odd for detail). In practice you may need to separate them afterwards

# idwt is esentially reversing the order of the lifting scheme steps in dwt and
# flipping the sign of operations(i.e. addition becomes subtraction and vice versa)

# approx_len = approximation coefficients length = ceil(total_series_len / 2)
# detail_len = detail coefficients length = floor(total_series_len / 2)

# If you do out the math, the equivalent 1D convolution filters are:
#
# approx analysis: (-1/8, 2/8, 6/8, 2/8, -1/8)
# detail analysis: (-1/2, 1, -1/2)
#
# approx synthesis: (1/2, 1, 1/2)
# detail synthesis: (-1/8, -2/8, 6/8, -2/8, -1/8)
#
# These don't actually work due to the weird use of rounding though.
# Le Gall seems designed specifically for use in a lifting scheme.

import numpy as np

def slow_dwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len

    for i in range(detail_len):
        seq[(i << 1) + 1] -= (seq[i << 1] + seq[min((i << 1) + 2, (approx_len - 1) << 1)]) >> 1
    
    for i in range(approx_len):
        seq[i << 1] += (seq[max((i << 1) - 1, 1)] + seq[min((i << 1) + 1, ((detail_len - 1) << 1) + 1)] + 2) >> 2

    return seq

def slow_idwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len
    
    for i in range(approx_len):
        seq[i << 1] -= (seq[max((i << 1) - 1, 1)] + seq[min((i << 1) + 1, ((detail_len - 1) << 1) + 1)] + 2) >> 2

    for i in range(detail_len):
        seq[(i << 1) + 1] += (seq[i << 1] + seq[min((i << 1) + 2, (approx_len - 1) << 1)]) >> 1
    
    return seq

def dwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len

    for i in range(1, detail_len):
        seq[(i << 1) - 1] -= (seq[(i - 1) << 1] + seq[i << 1]) >> 1
    seq[(detail_len << 1) - 1] -= (seq[(detail_len - 1) << 1] + seq[(approx_len - 1) << 1]) >> 1

    seq[0] += (seq[1] + seq[1] + 2) >> 2
    for i in range(1, approx_len - 1):
        seq[i << 1] += (seq[(i << 1) - 1] + seq[(i << 1) + 1] + 2) >> 2
    seq[(approx_len - 1) << 1] += (seq[(approx_len << 1) - 3] + seq[(detail_len << 1) - 1] + 2) >> 2

    return seq

def idwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len
    
    seq[0] -= (seq[1] + seq[1] + 2) >> 2
    for i in range(1, approx_len - 1):
        seq[i << 1] -= (seq[(i << 1) - 1] + seq[(i << 1) + 1] + 2) >> 2
    seq[(approx_len - 1) << 1] -= (seq[(approx_len << 1) - 3] + seq[(detail_len << 1) - 1] + 2) >> 2

    for i in range(1, detail_len):
        seq[(i << 1) - 1] += (seq[(i - 1) << 1] + seq[i << 1]) >> 1
    seq[(detail_len << 1) - 1] += (seq[(detail_len - 1) << 1] + seq[(approx_len - 1) << 1]) >> 1

    return seq

seq = np.arange(64)

print(seq)

coeffs = dwt(seq)
approx = coeffs[::2]
detail = coeffs[1::2]
print(f'\napprox: {approx}\ndetail: {detail}\n')
reconstructed = idwt(coeffs)
print(reconstructed)
if np.array_equal(seq, reconstructed):
    print("perfect reconstruction...")
# To-Do
# Perform 

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]

approx: [ 0  2  4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46
 48 50 52 54 56 58 60 62]
detail: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
perfect reconstruction...
